In [2]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split

In [3]:
# path_tomos= '/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/pictures_raw/train'
path_tomos= '/Volumes/DiskMorgane/bach_32/bach_1'
path_csv = '/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/csv_raw/train_labels.csv'

In [4]:
path_train_csv= '../data/csv_raw/train_labels.csv'


df = pd.read_csv(path_train_csv).copy()
df

# def generate_mean_patches(
#     tomo_root_path,
#     csv_path,
#     patch_size=512,
#     patches_per_tomo=32,  # Cambiado a 32 para coincidir con 1/32
#     num_tomos=None
# ):
#     """
#     Genera imágenes promedio por grupos de 1/32 del total de imágenes por tomo,
#     y devuelve los arrays junto con las etiquetas correspondientes.

#     Args:
#         tomo_root_path: Ruta a la carpeta que contiene las tomografías
#         csv_path: Ruta al archivo CSV con las etiquetas
#         patch_size: Tamaño de los parches a extraer (cuadrados)
#         patches_per_tomo: Número de parches promedio a generar por tomo (32)
#         num_tomos: Número máximo de tomografías a procesar (None para todas)

#     Returns:
#         tuple: (X, y)
#             - X: np.ndarray shape (N, 512, 512, 1), valores normalizados
#             - y: np.ndarray shape (N,), etiquetas (Number_of_motors)
#     """
#     all_patches = []
#     all_labels = []

#     # Cargar el DataFrame con la información
#     df = pd.read_csv(csv_path)

#     # Obtener lista de carpetas tipo tomo_*
#     tomo_dirs = sorted(glob(os.path.join(tomo_root_path, 'tomo_*')))
#     if num_tomos is not None:
#         tomo_dirs = tomo_dirs[:num_tomos]

#     for tomo_dir in tqdm(tomo_dirs, desc="Procesando tomos"):
#         tomo_id = os.path.basename(tomo_dir)

#         # Buscar la etiqueta correspondiente a este tomo
#         row = df[df['tomo_id'] == tomo_id]
#         if row.empty:
#             print(f"⚠️ No se encontró información en CSV para: {tomo_id}")
#             continue

#         # Etiqueta binaria: 1 si hay motores, 0 si no
#         num_motors = 1 if row['Number_of_motors'].values[0] > 0 else 0

#         # Cargar imágenes
#         image_paths = sorted(glob(os.path.join(tomo_dir, "*.png")) + glob(os.path.join(tomo_dir, "*.jpg")))
#         total_images = len(image_paths)

#         if total_images == 0:
#             print(f"⚠️ Tomo vacío: {tomo_id}")
#             continue

#         # Calcular tamaño de cada grupo (1/32 del total)
#         group_size = max(1, total_images // patches_per_tomo)

#         # Procesar en grupos
#         for group_idx in range(patches_per_tomo):
#             start_idx = group_idx * group_size
#             end_idx = start_idx + group_size if group_idx < patches_per_tomo - 1 else total_images

#             # Leer y promediar las imágenes del grupo actual
#             group_images = []
#             for img_path in image_paths[start_idx:end_idx]:
#                 img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#                 if img is not None:
#                     group_images.append(img.astype(np.float32))

#             if not group_images:
#                 continue

#             # Calcular imagen promedio del grupo
#             group_mean = np.mean(group_images, axis=0)
#             group_mean = cv2.normalize(group_mean, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#             # Extraer parche central de 512x512
#             h, w = group_mean.shape
#             if h < patch_size or w < patch_size:
#                 # Si la imagen es más pequeña que el patch_size, hacer padding
#                 pad_h = max(0, patch_size - h)
#                 pad_w = max(0, patch_size - w)
#                 group_mean = np.pad(group_mean,
#                                   ((0, pad_h), (0, pad_w)),
#                                   mode='constant')
#                 h, w = group_mean.shape

#             center_h = (h - patch_size) // 2
#             center_w = (w - patch_size) // 2
#             patch = group_mean[center_h:center_h+patch_size, center_w:center_w+patch_size]

#             all_patches.append(patch)
#             all_labels.append(num_motors)

#     # Convertir a arrays numpy
#     X = np.array(all_patches)
#     y = np.array(all_labels)

#     # Agregar dimensión de canal y normalizar
#     X = np.expand_dims(X, axis=-1)
#     X = X.astype(np.float32) / 255.0

#     return X, y

# X, y= generate_mean_patches(
#     tomo_root_path = path_tomos,
#     csv_path = path_csv,
#     patch_size=512,
#     patches_per_tomo=32,
#     num_tomos=628
#     )


,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0
1,1,30,tomo_098751,-1.0,-1.0,-1.0,500,924,956,16.1,0
2,2,54,tomo_136c8d,-1.0,-1.0,-1.0,500,924,956,19.7,0
3,3,59,tomo_146de2,-1.0,-1.0,-1.0,500,924,956,16.1,0
4,4,84,tomo_1dc5f9,-1.0,-1.0,-1.0,500,924,956,19.7,0
...,...,...,...,...,...,...,...,...,...,...,...
732,732,99,tomo_226cd8,366.0,693.0,269.0,500,928,960,13.1,10
733,733,100,tomo_226cd8,386.0,693.0,319.0,500,928,960,13.1,10
734,734,101,tomo_226cd8,388.0,671.0,433.0,500,928,960,13.1,10
735,735,102,tomo_226cd8,411.0,318.0,740.0,500,928,960,13.1,10


,tomo_id,Unnamed: 0,row_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors,motor_place
1,tomo_00e047,338,1,169.0,546.0,603.0,300,959,928,15.6,1,18
38,tomo_0fe63f,408,45,197.0,362.0,265.0,300,960,928,13.1,1,21


In [ ]:
print ( X.shape), print ( y.shape)

In [ ]:
random_state=42

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.05,
                                                    random_state=random_state)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.2,
                                                  random_state=random_state)

In [ ]:

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_addons.metrics import FBetaScore
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
# from tensorflow.keras.metrics import Recall, Precision


def init_model_densenet(X):
    original_input_shape = X.shape[1:]

    #  force input to 3 channel if it's grey (1 canal)
    if original_input_shape[-1] == 1:
        base_input_shape = original_input_shape[:-1] + (3,)
    else:
        base_input_shape = original_input_shape

    # load DenseNet121 without last classify layer
    base_model = DenseNet121(include_top=False,
                             weights='imagenet',
                             input_shape=base_input_shape
                             )
    base_model.trainable = False

    # Input layer
    inputs = Input(shape=original_input_shape)

    # convert in RGB if need it,
    if original_input_shape[-1] == 1:
        x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(inputs)
    else:
        x = inputs

    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-4),
        metrics=[
            'accuracy',
            FBetaScore(num_classes=1,
                       average='micro',
                       beta=2.0),
            Recall(),
            Precision(),
        ]
    )

    return model


def model_train (model, X_train_processed, y_train, batch_size=16 ,epochs=5):
    verbose=1
    validation_split=0.1
    es = EarlyStopping(patience=3,
                       restore_best_weights=True)

    history = model.fit(X_train_processed,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        # validation_split=validation_split,
                        validation_data=(X_val, y_val),
                        callbacks=[es],
                        verbose=verbose)
    return history


In [ ]:
model = init_model_densenet(X_train)

In [ ]:

history = model_train(model, X_train,y_train)
history

In [ ]:
history.history

In [ ]:
def plot_training_metrics(history):
    """
   plot the metrics, even if there is not one of them into 'history'
    """
    history_dict = history.history
    plotted = False

    def safe_plot(metric_name, color, linestyle='-'):
        nonlocal plotted
        if metric_name in history_dict:
            label = metric_name.replace('_', ' ')
            plt.plot(history_dict[metric_name], c=color, linestyle=linestyle, label=label)
            plotted = True

    # fbeta
    safe_plot('fbeta_score', 'r')
    safe_plot('val_fbeta_score', 'r', '--')

    # recall
    safe_plot('recall', 'g')
    safe_plot('val_recall', 'g', '--')
    safe_plot('recall_1', 'g')  # alternativa por si usa nombre automático
    safe_plot('val_recall_1', 'g', '--')

    # accuracy
    safe_plot('accuracy', 'black')
    safe_plot('val_accuracy', 'black', '--')

    # precision
    safe_plot('precision', 'blue')
    safe_plot('val_precision', 'blue', '--')

    if plotted:
        plt.title("Training Metrics")
        plt.xlabel("Epoch")
        plt.ylabel("Metric Value")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("⚠️ No se encontró ninguna métrica conocida para graficar.")


plot_training_metrics(history)